In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import os
import cv2 # Using OpenCV for image loading/resizing
from tqdm.notebook import tqdm # For progress bars

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Ensure you have a GPU available for faster training
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if not tf.config.experimental.list_physical_devices('GPU'):
    print("WARNING: No GPU found. Training will be extremely slow on CPU. Please enable GPU runtime: Runtime -> Change runtime type -> GPU.")

# ----------------------------------------------------------------------
# Step 1: Mount Google Drive (if you haven't already in this session)
# ----------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------------------------------------------------
# Step 2: Load and Prepare Your Full Dataset
# ----------------------------------------------------------------------
# --- IMPORTANT: Adjust this path to your combined dataset folder ---
data_dir = '/content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/combined_dataset_for_kfold'

image_paths = []
labels = []

# Assuming your class folders are directly inside data_dir
class_names = sorted(os.listdir(data_dir))
class_to_idx = {name: i for i, name in enumerate(class_names)}

print(f"Detected classes: {class_names}")

for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path): # Ensure it's a directory
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if os.path.isfile(img_path): # Ensure it's a file
                image_paths.append(img_path)
                labels.append(class_to_idx[class_name])
    else:
        print(f"Warning: '{class_path}' is not a directory. Skipping.")

image_paths = np.array(image_paths)
labels = np.array(labels)

print(f"\nTotal number of samples loaded for K-fold: {len(image_paths)}")
print(f"Initial Class distribution: {np.bincount(labels)}")

# Define input shape for InceptionV3
IMG_HEIGHT, IMG_WIDTH = 299, 299 # InceptionV3 typically uses 299x299
NUM_CLASSES = len(class_names)
BATCH_SIZE = 32 # Adjust as needed based on GPU memory

# Data preprocessing function (loading, resizing, normalization)
# This will be mapped to the tf.data.Dataset
def load_and_preprocess_image(image_path, label, img_size=(IMG_HEIGHT, IMG_WIDTH)):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = preprocess_input(img) # InceptionV3 specific preprocessing
    label = tf.one_hot(label, NUM_CLASSES) # Convert to one-hot encoding
    return img, label

# Data augmentation function (to be used in tf.data.Dataset map for training)
def augment_image(image, label):
    # Apply your augmentations here (based on your previous oversampling strategy)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # The following lines were causing the AttributeError and are commented out:
    # image = tf.image.random_zoom(image, zoom_factor=(0.8, 1.2)) # Example zoom
    # image = tf.image.random_rotation(image, factor=0.1) # Example rotation

    # If you still want these augmentations, consider using:
    # - tf.keras.layers.RandomRotation, RandomZoom, etc. applied after batching
    # - Or updating your TensorFlow version (e.g., to 2.3+)

    return image, label

# ----------------------------------------------------------------------
# Step 3: Define Your Model Building Function
# ----------------------------------------------------------------------
def build_inceptionv3_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers (optional, but common for transfer learning)
    # You can unfreeze some layers for fine-tuning after initial training if desired
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x) # Increased units slightly from previous example for potential improvement
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ----------------------------------------------------------------------
# Step 4: Implement Your Hybrid Balancing Strategy (within the fold)
# ----------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import class_weight

def apply_hybrid_balancing_to_fold(X_train_paths, y_train_labels, class_names):
    # 1. Undersampling on the majority class
    # Use the indices for sampling to apply back to paths and labels
    indices = np.arange(len(X_train_paths)).reshape(-1, 1) # Dummy features for imblearn

    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    resampled_indices_dummy, y_resampled_labels = rus.fit_resample(indices, y_train_labels)

    X_resampled_paths = X_train_paths[resampled_indices_dummy.flatten()]

    # 2. Class Weights (implicitly handles "oversampling" in terms of loss contribution)
    # These weights tell the loss function to give more importance to the minority class samples
    unique_classes = np.unique(y_resampled_labels)
    # Ensure correct ordering of classes if needed, for class_weight
    weights = class_weight.compute_class_weight(class_weight='balanced',
                                                classes=np.array(range(len(class_names))), # Use all possible class indices
                                                y=y_resampled_labels)
    class_weights_dict = dict(zip(range(len(class_names)), weights))

    return X_resampled_paths, y_resampled_labels, class_weights_dict

# ----------------------------------------------------------------------
# Step 5: K-Fold Cross-Validation Loop
# ----------------------------------------------------------------------
N_SPLITS = 5 # Using 5 folds is a good balance between computational cost and statistical robustness
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

fold_results = [] # To store metrics for each fold

for fold_idx, (train_index, val_index) in enumerate(skf.split(image_paths, labels)):
    print(f"\n--- Starting Fold {fold_idx + 1}/{N_SPLITS} ---")

    # Split data for the current fold
    X_train_fold_paths, X_val_fold_paths = image_paths[train_index], image_paths[val_index]
    y_train_fold_labels, y_val_fold_labels = labels[train_index], labels[val_index]

    print(f"  Train samples in fold: {len(X_train_fold_paths)}")
    print(f"  Validation samples in fold: {len(X_val_fold_paths)}")
    print(f"  Train labels distribution (before balancing): {np.bincount(y_train_fold_labels)}")
    print(f"  Val labels distribution: {np.bincount(y_val_fold_labels)}")

    # Apply Hybrid Balancing ONLY to the training data of the current fold
    X_train_balanced_paths, y_train_balanced_labels, class_weights = \
        apply_hybrid_balancing_to_fold(X_train_fold_paths, y_train_fold_labels, class_names)

    print(f"  Train samples AFTER balancing (undersampled): {len(X_train_balanced_paths)}")
    print(f"  Balanced Train labels distribution: {np.bincount(y_train_balanced_labels)}")
    print(f"  Class Weights for training: {class_weights}")


    # Create tf.data.Dataset for training and validation
    # Training Dataset: Load, Augment, Shuffle, Batch, Prefetch
    train_ds = tf.data.Dataset.from_tensor_slices((X_train_balanced_paths, y_train_balanced_labels))
    train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE) # Apply augmentation
    train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Validation Dataset: Load, Batch, Prefetch (no augmentation for validation)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val_fold_paths, y_val_fold_labels))
    val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Build and compile a fresh model for each fold
    model = build_inceptionv3_model()

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True), # Increased patience
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001, verbose=1) # Increased patience
    ]

    # Train the model
    print("  Training model...")
    history = model.fit(train_ds,
                        epochs=100, # Max epochs, EarlyStopping will stop it
                        validation_data=val_ds,
                        class_weight=class_weights, # Apply class weights during training
                        callbacks=callbacks,
                        verbose=1)

    # Evaluate the model on the validation data for the current fold
    print("  Evaluating model on validation set...")
    y_true_val = []
    y_pred_proba_val = []

    # Iterate through the validation dataset to collect true labels and predictions
    # Use tqdm for progress bar if validation set is large
    for images, labels_one_hot in tqdm(val_ds, desc="Predicting on validation set"):
        y_true_val.extend(np.argmax(labels_one_hot.numpy(), axis=1))
        y_pred_proba_val.extend(model.predict(images))

    y_pred_val = np.argmax(np.array(y_pred_proba_val), axis=1)

    # Calculate overall metrics for the current fold
    fold_accuracy = accuracy_score(y_true_val, y_pred_val)

    # Get detailed per-class metrics using classification_report
    report = classification_report(y_true_val, y_pred_val, target_names=class_names, output_dict=True, zero_division=0)

    # Determine the minority class from the initial data distribution
    minority_class_label_idx = np.bincount(labels).argmin()
    minority_class_name = class_names[minority_class_label_idx]

    minority_precision = report[minority_class_name]['precision']
    minority_recall = report[minority_class_name]['recall']
    minority_f1 = report[minority_class_name]['f1-score']

    print(f"Fold {fold_idx + 1} Metrics:")
    print(f"  Overall Accuracy: {fold_accuracy:.4f}")
    print(f"  Minority Class ({minority_class_name}):")
    print(f"    Precision: {minority_precision:.4f}")
    print(f"    Recall: {minority_recall:.4f}")
    print(f"    F1-score: {minority_f1:.4f}")
    print("  Full Classification Report:\n", classification_report(y_true_val, y_pred_val, target_names=class_names, zero_division=0))

    fold_results.append({
        'overall_accuracy': fold_accuracy,
        'minority_precision': minority_precision,
        'minority_recall': minority_recall,
        'minority_f1': minority_f1
    })

    # Clear session to free up memory before next fold
    tf.keras.backend.clear_session()
    del model # Delete model instance to release resources

# ----------------------------------------------------------------------
# Step 6: Aggregate and Report Final Results
# ----------------------------------------------------------------------
print("\n--- K-Fold Cross-Validation Final Results ---")
avg_overall_accuracy = np.mean([res['overall_accuracy'] for res in fold_results])
std_overall_accuracy = np.std([res['overall_accuracy'] for res in fold_results])

avg_minority_precision = np.mean([res['minority_precision'] for res in fold_results])
std_minority_precision = np.std([res['minority_precision'] for res in fold_results])

avg_minority_recall = np.mean([res['minority_recall'] for res in fold_results])
std_minority_recall = np.std([res['minority_recall'] for res in fold_results])

avg_minority_f1 = np.mean([res['minority_f1'] for res in fold_results])
std_minority_f1 = np.std([res['minority_f1'] for res in fold_results])

print(f"Average Overall Accuracy: {avg_overall_accuracy:.4f} +/- {std_overall_accuracy:.4f}")
print(f"Average Minority Precision: {avg_minority_precision:.4f} +/- {std_minority_precision:.4f}")
print(f"Average Minority Recall: {avg_minority_recall:.4f} +/- {std_minority_recall:.4f}")
print(f"Average Minority F1-score: {avg_minority_f1:.4f} +/- {std_minority_f1:.4f}")

Num GPUs Available:  1
Mounted at /content/drive
Detected classes: ['Arctocephalus_galapagoensis', 'Zalophus_wollebaeki']

Total number of samples loaded for K-fold: 336
Initial Class distribution: [ 45 291]

--- Starting Fold 1/5 ---
  Train samples in fold: 268
  Validation samples in fold: 68
  Train labels distribution (before balancing): [ 36 232]
  Val labels distribution: [ 9 59]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class Weights for training: {0: np.float64(1.0), 1: np.float64(1.0)}
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
  Training model...
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 209s 15s/step - accuracy: 0.4106 - loss: 1.0694 - val_accuracy: 0.1471 - val_loss: 2.6856 - learning_rate: 0.0010
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 441ms/step - accuracy: 0.5655 - loss: 1.1505 - val_accuracy: 0.9118 - val_loss: 0.2295 - learning_rate: 0.0010
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 532ms/step - accuracy: 0.6662

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Fold 1 Metrics:
  Overall Accuracy: 0.9706
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.8182
    Recall: 1.0000
    F1-score: 0.9000
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.82      1.00      0.90         9
        Zalophus_wollebaeki       1.00      0.97      0.98        59

                   accuracy                           0.97        68
                  macro avg       0.91      0.98      0.94        68
               weighted avg       0.98      0.97      0.97        68


--- Starting Fold 2/5 ---
  Train samples in fold: 269
  Validation samples in fold: 67
  Train labels distribution (before balancing): [ 36 233]
  Val labels distribution: [ 9 58]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class W

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Fold 2 Metrics:
  Overall Accuracy: 0.9552
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.8000
    Recall: 0.8889
    F1-score: 0.8421
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.80      0.89      0.84         9
        Zalophus_wollebaeki       0.98      0.97      0.97        58

                   accuracy                           0.96        67
                  macro avg       0.89      0.93      0.91        67
               weighted avg       0.96      0.96      0.96        67


--- Starting Fold 3/5 ---
  Train samples in fold: 269
  Validation samples in fold: 67
  Train labels distribution (before balancing): [ 36 233]
  Val labels distribution: [ 9 58]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class W

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Fold 3 Metrics:
  Overall Accuracy: 0.9552
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.8000
    Recall: 0.8889
    F1-score: 0.8421
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.80      0.89      0.84         9
        Zalophus_wollebaeki       0.98      0.97      0.97        58

                   accuracy                           0.96        67
                  macro avg       0.89      0.93      0.91        67
               weighted avg       0.96      0.96      0.96        67


--- Starting Fold 4/5 ---
  Train samples in fold: 269
  Validation samples in fold: 67
  Train labels distribution (before balancing): [ 36 233]
  Val labels distribution: [ 9 58]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class Weights for training: {0: np.float64(1.0), 1: np.float64(1.0)}
  Training mo

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Fold 4 Metrics:
  Overall Accuracy: 0.9403
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.7778
    Recall: 0.7778
    F1-score: 0.7778
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.78      0.78      0.78         9
        Zalophus_wollebaeki       0.97      0.97      0.97        58

                   accuracy                           0.94        67
                  macro avg       0.87      0.87      0.87        67
               weighted avg       0.94      0.94      0.94        67


--- Starting Fold 5/5 ---
  Train samples in fold: 269
  Validation samples in fold: 67
  Train labels distribution (before balancing): [ 36 233]
  Val labels distribution: [ 9 58]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class W

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Fold 5 Metrics:
  Overall Accuracy: 0.9254
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.7500
    Recall: 0.6667
    F1-score: 0.7059
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.75      0.67      0.71         9
        Zalophus_wollebaeki       0.95      0.97      0.96        58

                   accuracy                           0.93        67
                  macro avg       0.85      0.82      0.83        67
               weighted avg       0.92      0.93      0.92        67


--- K-Fold Cross-Validation Final Results ---
Average Overall Accuracy: 0.9493 +/- 0.0153
Average Minority Precision: 0.7892 +/- 0.0234
Average Minority Recall: 0.8444 +/- 0.1133
Average Minority F1-score: 0.8136 +/- 0.0663


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix # Added confusion_matrix
import os
import cv2
from tqdm.notebook import tqdm
import pandas as pd
import json

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Ensure you have a GPU available for faster training
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if not tf.config.experimental.list_physical_devices('GPU'):
    print("WARNING: No GPU found. Training will be extremely slow on CPU. Please enable GPU runtime: Runtime -> Change runtime type -> GPU.")

# ----------------------------------------------------------------------
# Step 1: Mount Google Drive (if not already mounted)
# ----------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------------------------------------------------
# Step 2: Load and Prepare Your Full Dataset
# ----------------------------------------------------------------------
# --- IMPORTANT: This path should point to your combined dataset folder ---
data_dir = '/content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/combined_dataset_for_kfold'

image_paths = []
labels = []

class_names = sorted(os.listdir(data_dir))
class_to_idx = {name: i for i, name in enumerate(class_names)}

print(f"Detected classes: {class_names}")

for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(class_to_idx[class_name])
    else:
        print(f"Warning: '{class_path}' is not a directory. Skipping.")

image_paths = np.array(image_paths)
labels = np.array(labels)

print(f"\nTotal number of samples loaded for K-fold: {len(image_paths)}")
print(f"Initial Class distribution: {np.bincount(labels)}")

# Define input shape for InceptionV3
IMG_HEIGHT, IMG_WIDTH = 299, 299
NUM_CLASSES = len(class_names)
BATCH_SIZE = 32

# Data preprocessing function
def load_and_preprocess_image(image_path, label, img_size=(IMG_HEIGHT, IMG_WIDTH)):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = preprocess_input(img)
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

# Data augmentation function
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

# ----------------------------------------------------------------------
# Step 3: Define Your Model Building Function
# ----------------------------------------------------------------------
def build_inceptionv3_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ----------------------------------------------------------------------
# Step 4: Implement Your Hybrid Balancing Strategy (within the fold)
# ----------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import class_weight

def apply_hybrid_balancing_to_fold(X_train_paths, y_train_labels, class_names):
    indices = np.arange(len(X_train_paths)).reshape(-1, 1)
    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    resampled_indices_dummy, y_resampled_labels = rus.fit_resample(indices, y_train_labels)
    X_resampled_paths = X_train_paths[resampled_indices_dummy.flatten()]

    unique_classes = np.unique(y_resampled_labels)
    weights = class_weight.compute_class_weight(class_weight='balanced',
                                                classes=np.array(range(len(class_names))),
                                                y=y_resampled_labels)
    class_weights_dict = dict(zip(range(len(class_names)), weights))

    return X_resampled_paths, y_resampled_labels, class_weights_dict

# ----------------------------------------------------------------------
# Define where to save K-fold results
# This will create a 'kfold_results' folder inside 'my_galapagos_seals_dataset'
# ----------------------------------------------------------------------
kfold_results_base_dir = os.path.join(os.path.dirname(data_dir), 'kfold_results')
os.makedirs(kfold_results_base_dir, exist_ok=True)
print(f"\nK-fold results will be saved in: {kfold_results_base_dir}")

# ----------------------------------------------------------------------
# Step 5: K-Fold Cross-Validation Loop (MODIFIED for saving results & CM)
# ----------------------------------------------------------------------
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

fold_results = []
all_confusion_matrices = [] # List to store confusion matrix for each fold

for fold_idx, (train_index, val_index) in enumerate(skf.split(image_paths, labels)):
    print(f"\n--- Starting Fold {fold_idx + 1}/{N_SPLITS} ---")

    # Define and create directory for current fold's results
    current_fold_dir = os.path.join(kfold_results_base_dir, f'fold_{fold_idx}')
    os.makedirs(current_fold_dir, exist_ok=True)
    print(f"  Results for Fold {fold_idx + 1} will be saved in: {current_fold_dir}")

    # Split data for the current fold
    X_train_fold_paths, X_val_fold_paths = image_paths[train_index], image_paths[val_index]
    y_train_fold_labels, y_val_fold_labels = labels[train_index], labels[val_index]

    print(f"  Train samples in fold: {len(X_train_fold_paths)}")
    print(f"  Validation samples in fold: {len(X_val_fold_paths)}")
    print(f"  Train labels distribution (before balancing): {np.bincount(y_train_fold_labels)}")
    print(f"  Val labels distribution: {np.bincount(y_val_fold_labels)}")

    # Apply Hybrid Balancing ONLY to the training data of the current fold
    X_train_balanced_paths, y_train_balanced_labels, class_weights = \
        apply_hybrid_balancing_to_fold(X_train_fold_paths, y_train_fold_labels, class_names)

    print(f"  Train samples AFTER balancing (undersampled): {len(X_train_balanced_paths)}")
    print(f"  Balanced Train labels distribution: {np.bincount(y_train_balanced_labels)}")
    print(f"  Class Weights for training: {class_weights}")

    # Create tf.data.Dataset for training and validation
    train_ds = tf.data.Dataset.from_tensor_slices((X_train_balanced_paths, y_train_balanced_labels))
    train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices((X_val_fold_paths, y_val_fold_labels))
    val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Build and compile a fresh model for each fold
    model = build_inceptionv3_model()

    # Callbacks (with ModelCheckpoint added)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(current_fold_dir, 'best_model.h5'),
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001, verbose=1),
        model_checkpoint_callback
    ]

    # Train the model
    print("  Training model...")
    history = model.fit(train_ds,
                        epochs=100,
                        validation_data=val_ds,
                        class_weight=class_weights,
                        callbacks=callbacks,
                        verbose=1)

    # Save training history for the current fold
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(current_fold_dir, 'training_history.csv'), index=False)
    print(f"  Saved training history for Fold {fold_idx + 1}.")

    # Evaluate the model on the validation data for the current fold
    print("  Evaluating model on validation set...")
    y_true_val = []
    y_pred_proba_val = []

    for images, labels_one_hot in tqdm(val_ds, desc="Predicting on validation set"):
        y_true_val.extend(np.argmax(labels_one_hot.numpy(), axis=1))
        y_pred_proba_val.extend(model.predict(images))

    y_pred_val = np.argmax(np.array(y_pred_proba_val), axis=1)

    # Calculate metrics for the current fold
    fold_accuracy = accuracy_score(y_true_val, y_pred_val)
    report = classification_report(y_true_val, y_pred_val, target_names=class_names, output_dict=True, zero_division=0)

    # Calculate Confusion Matrix for the current fold
    cm = confusion_matrix(y_true_val, y_pred_val)
    all_confusion_matrices.append(cm) # Store for averaging later

    print(f"\nFold {fold_idx + 1} Metrics:")
    print(f"  Overall Accuracy: {fold_accuracy:.4f}")

    minority_class_label_idx = np.bincount(labels).argmin()
    minority_class_name = class_names[minority_class_label_idx]

    minority_precision = report[minority_class_name]['precision']
    minority_recall = report[minority_class_name]['recall']
    minority_f1 = report[minority_class_name]['f1-score']

    print(f"  Minority Class ({minority_class_name}):")
    print(f"    Precision: {minority_precision:.4f}")
    print(f"    Recall: {minority_recall:.4f}")
    print(f"    F1-score: {minority_f1:.4f}")
    print("  Full Classification Report:\n", classification_report(y_true_val, y_pred_val, target_names=class_names, zero_division=0))

    # Print Confusion Matrix for the current fold (formatted with pandas for clarity)
    print("\n  Confusion Matrix for Fold {}:".format(fold_idx + 1))
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    print(cm_df)

    # Save Confusion Matrix for the current fold
    cm_filepath = os.path.join(current_fold_dir, 'confusion_matrix.csv')
    cm_df.to_csv(cm_filepath)
    print(f"  Confusion Matrix saved to: {cm_filepath}")

    fold_results.append({
        'overall_accuracy': fold_accuracy,
        'minority_precision': minority_precision,
        'minority_recall': minority_recall,
        'minority_f1': minority_f1,
        'confusion_matrix': cm.tolist() # Convert numpy array to list for JSON compatibility
    })

    tf.keras.backend.clear_session()
    del model

# ----------------------------------------------------------------------
# Step 6: Aggregate and Report Final Results (MODIFIED to include Average CM)
# ----------------------------------------------------------------------
print("\n--- K-Fold Cross-Validation Final Results ---")
avg_overall_accuracy = np.mean([res['overall_accuracy'] for res in fold_results])
std_overall_accuracy = np.std([res['overall_accuracy'] for res in fold_results])

avg_minority_precision = np.mean([res['minority_precision'] for res in fold_results])
std_minority_precision = np.std([res['minority_precision'] for res in fold_results])

avg_minority_recall = np.mean([res['minority_recall'] for res in fold_results])
std_minority_recall = np.std([res['minority_recall'] for res in fold_results])

avg_minority_f1 = np.mean([res['minority_f1'] for res in fold_results])
std_minority_f1 = np.std([res['minority_f1'] for res in fold_results])

# Calculate and display Average Confusion Matrix
# np.array(all_confusion_matrices) converts list of CMs to a 3D numpy array
# axis=0 means average across the first dimension (the folds)
average_cm = np.mean(np.array(all_confusion_matrices), axis=0)
print("\n--- Average Confusion Matrix across all Folds (Rounded to 2 decimal places) ---")
average_cm_df = pd.DataFrame(average_cm, index=class_names, columns=class_names)
print(average_cm_df.round(2)) # Round for cleaner display of averages

final_summary = {
    "Average Overall Accuracy": f"{avg_overall_accuracy:.4f} +/- {std_overall_accuracy:.4f}",
    "Average Minority Precision": f"{avg_minority_precision:.4f} +/- {std_minority_precision:.4f}",
    "Average Minority Recall": f"{avg_minority_recall:.4f} +/- {std_minority_recall:.4f}",
    "Average Minority F1-score": f"{avg_minority_f1:.4f} +/- {std_minority_f1:.4f}",
    "Average Confusion Matrix (rows are true, columns are predicted)": average_cm.round(2).tolist()
}

print(json.dumps(final_summary, indent=4))

# Save the final summary to a text file (JSON format for readability)
summary_filepath = os.path.join(kfold_results_base_dir, 'kfold_summary_metrics.json')
with open(summary_filepath, 'w') as f:
    json.dump(final_summary, f, indent=4)
print(f"\nFinal K-fold summary metrics and average CM saved to: {summary_filepath}")

Num GPUs Available:  1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Detected classes: ['Arctocephalus_galapagoensis', 'Zalophus_wollebaeki']

Total number of samples loaded for K-fold: 336
Initial Class distribution: [ 45 291]

K-fold results will be saved in: /content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/kfold_results

--- Starting Fold 1/5 ---
  Results for Fold 1 will be saved in: /content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/kfold_results/fold_0
  Train samples in fold: 268
  Validation samples in fold: 68
  Train labels distribution (before balancing): [ 36 232]
  Val labels distribution: [ 9 59]
  Train samples AFTER balancing (undersampled): 72
  Balanced Train labels distribution: [36 36]
  Class Weights for training: {0: np.float64(1.0), 1: np.float64(1.0)}
  Training model...
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 31s 9s/step - accuracy: 0.5482 - loss: 1.2630 - val_accuracy: 0.1618 - val_loss: 2.5557 - learning_rate: 0.0010
Epoch 2/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.4766 - loss: 1.3043
Epoch 2: val_loss improved from 2.55570 to 0.22197, saving model to /content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/kfold_results/fold_0/best_model.h5


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 858ms/step - accuracy: 0.5438 - loss: 1.1186 - val_accuracy: 0.9412 - val_loss: 0.2220 - learning_rate: 0.0010
Epoch 3/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.7734 - loss: 0.3818
Epoch 3: val_loss did not improve from 0.22197
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 411ms/step - accuracy: 0.7478 - loss: 0.4058 - val_accuracy: 0.7941 - val_loss: 0.3833 - learning_rate: 0.0010
Epoch 4/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.8984 - loss: 0.2296
Epoch 4: val_loss did not improve from 0.22197
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 467ms/step - accuracy: 0.8867 - loss: 0.2436 - val_accuracy: 0.5294 - val_loss: 1.0411 - learning_rate: 0.0010
Epoch 5/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.8516 - loss: 0.2969
Epoch 5: val_loss did not improve from 0.22197
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 368ms/step - accuracy: 0.8633 - loss: 0.2904 - val_accuracy: 0.8235 - val_loss: 0.2966 - learning_rate: 0.0010
Epoch 6/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/st

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9813 - loss: 0.1340 - val_accuracy: 0.9412 - val_loss: 0.1496 - learning_rate: 0.0010
Epoch 7/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.9766 - loss: 0.1302
Epoch 7: val_loss did not improve from 0.14960
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 414ms/step - accuracy: 0.9813 - loss: 0.1322 - val_accuracy: 0.8382 - val_loss: 0.2558 - learning_rate: 0.0010
Epoch 8/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.9922 - loss: 0.0836
Epoch 8: val_loss did not improve from 0.14960
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step - accuracy: 0.9891 - loss: 0.0808 - val_accuracy: 0.8088 - val_loss: 0.4343 - learning_rate: 0.0010
Epoch 9/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.9766 - loss: 0.0774
Epoch 9: val_loss did not improve from 0.14960
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 412ms/step - accuracy: 0.9813 - loss: 0.0748 - val_accuracy: 0.8235 - val_loss: 0.3512 - learning_rate: 0.0010
Epoch 10/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 881ms/step - accuracy: 0.9705 - loss: 0.0813 - val_accuracy: 0.9254 - val_loss: 0.1624 - learning_rate: 0.0010
Epoch 8/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 1.0000 - loss: 0.0650
Epoch 8: val_loss did not improve from 0.16237
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 729ms/step - accuracy: 1.0000 - loss: 0.0646 - val_accuracy: 0.8955 - val_loss: 0.2010 - learning_rate: 0.0010
Epoch 9/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 1.0000 - loss: 0.0298
Epoch 9: val_loss did not improve from 0.16237
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 389ms/step - accuracy: 1.0000 - loss: 0.0334 - val_accuracy: 0.8955 - val_loss: 0.3164 - learning_rate: 0.0010
Epoch 10/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 1.0000 - loss: 0.0330
Epoch 10: val_loss did not improve from 0.16237
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 363ms/step - accuracy: 1.0000 - loss: 0.0340 - val_accuracy: 0.9104 - val_loss: 0.2193 - learning_rate: 0.0010
Epoch 11/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 834ms/step - accuracy: 1.0000 - loss: 0.0177 - val_accuracy: 0.8955 - val_loss: 0.1571 - learning_rate: 0.0010
Epoch 12/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 1.0000 - loss: 0.0194
Epoch 12: val_loss improved from 0.15707 to 0.15260, saving model to /content/drive/My Drive/galapagos_seals_annotated_data/my_galapagos_seals_dataset/kfold_results/fold_4/best_model.h5


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 862ms/step - accuracy: 1.0000 - loss: 0.0189 - val_accuracy: 0.9104 - val_loss: 0.1526 - learning_rate: 0.0010
Epoch 13/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 1.0000 - loss: 0.0128
Epoch 13: val_loss did not improve from 0.15260
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 488ms/step - accuracy: 1.0000 - loss: 0.0137 - val_accuracy: 0.8955 - val_loss: 0.1727 - learning_rate: 0.0010
Epoch 14/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.0134
Epoch 14: val_loss did not improve from 0.15260
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 726ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 0.9104 - val_loss: 0.2307 - learning_rate: 0.0010
Epoch 15/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 1.0000 - loss: 0.0070
Epoch 15: val_loss did not improve from 0.15260
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 359ms/step - accuracy: 1.0000 - loss: 0.0083 - val_accuracy: 0.8955 - val_loss: 0.2679 - learning_rate: 0.0010
Epoch 16/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1

Predicting on validation set:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

Fold 5 Metrics:
  Overall Accuracy: 0.9104
  Minority Class (Arctocephalus_galapagoensis):
    Precision: 0.6364
    Recall: 0.7778
    F1-score: 0.7000
  Full Classification Report:
                              precision    recall  f1-score   support

Arctocephalus_galapagoensis       0.64      0.78      0.70         9
        Zalophus_wollebaeki       0.96      0.93      0.95        58

                   accuracy                           0.91        67
                  macro avg       0.80      0.85      0.82        67
               weighted avg       0.92      0.91      0.91        67


  Confusion Matrix for Fold 5:
                             Arctocephalus_galapagoensis  Zalophus_wollebaeki
Arctocephalus_galapagoensis                            7                    2
Zalophus_wollebaeki                                    4                   54
  Confusion Matrix sa